# Urban Sound Narrative

A Project that turns the day to day sound to a beautiful narration 

### Installing Dependencies

In [21]:
# For google collab:

# !pip install --quiet librosa torch torchaudio gtts ipywidgets

# Local Jupyter Notebook

%pip install --quiet librosa torch torchaudio gtts ipywidgets

Note: you may need to restart the kernel to use updated packages.


## Importing Libraries

In [22]:
import librosa
import torch
import torchaudio
from gtts import gTTS
import ipywidgets as widgets 
from IPython.display import display, Audio
import os
import time

In [23]:
upload_button = widgets.FileUpload(accept='.wav, .mp3', multiple=False)

In [24]:
def preprocess(audio_path, target_sr=22050, top_db=25):
    y, sr = librosa.load(audio_path, sr=target_sr)
    if y.ndim > 1:
        y = librosa.to_mono(y)
    y = librosa.util.normalize(y)
    return y, sr

In [25]:
def detect_sounds(waveform, sample_rate):
    # Calculate audio duration
    duration = librosa.get_duration(y=waveform, sr=sample_rate)
    
    # Mock sound detection based on duration
    sound_list = []
    if duration < 10:
        sound_list = ["car horns", "footsteps"]
    else:
        sound_list = ["car horns", "footsteps", "chattering", "dog barking"]
    
    print(f"Detected sounds (mock): {sound_list}")
    return sound_list

In [26]:
output = widgets.Output()

def on_upload_change(change):
    with output:
        output.clear_output()
        try:
            # Get the uploaded file
            uploaded_file = change['new'][0]
            
            # Generate a unique file path
            timestamp = str(int(time.time()))
            file_path = os.path.join('data', 'raw', f'audio_{timestamp}.wav')
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            
            # Save the file
            with open(file_path, 'wb') as f:
                f.write(uploaded_file['content'])
            
            # Preprocess the audio
            y, sr = preprocess(file_path)
            
            # Detect sounds
            sounds = detect_sounds(y, sr)
            
            # Display the output
            print(f"Audio loaded and preprocessed. Sample rate: {sr}, Shape: {y.shape}")
            display(Audio(y, rate=sr))  # Audio output
            print(f"Detected sounds: {sounds}")
            
            # Store globally for later use
            global waveform, sample_rate
            waveform, sample_rate = y, sr
            
        except Exception as e:
            print(f"Error processing audio: {str(e)}")

# Display widgets and link the handler
display(upload_button, output)
upload_button.observe(on_upload_change, names='value')

FileUpload(value=(), accept='.wav, .mp3', description='Upload')

Output()